In [1]:
# List the current work directory
import os
os.chdir('/home/huyenngn/Documents/oanquan')

In [2]:


os.getcwd()

'/home/huyenngn/Documents/oanquan'

In [3]:
from oanquan import api, oanquan
from copy import deepcopy
import random
import csv
import json

index = 30

In [4]:

def create_challenge(ideal_moves: int = 3):
    """Create a O An Quan challenge."""
    while True:
        while True:
            random_state = oanquan.OAnQuan.start_game()
            for _ in range(random.randint(5, 10)):
                api.make_random_move(random_state)
                if random_state.check_end():
                    break
            if not random_state.check_end():
                break
        if random_state.get_current_player() == oanquan.Player.COMPUTER:
            api.make_random_move(random_state)
        if random_state.check_end():
            continue
        game = deepcopy(random_state)
        states = [game.model_dump()]
        while not game.check_end():
            api.make_ab_move(game)
            states.append(game.model_dump())
        if game.get_winner() != oanquan.Player.PLAYER.name:
            continue
        if len(states) < 2:
            continue
        factor = (len(states) - ideal_moves*2)//2
        if factor > 0:
            random_state = states[factor*2]
        else:
            random_state = random_state.model_dump()
            ideal_moves = len(states)//2
        break
        
    return {
        "game": random_state,
        "hint": api.get_hint(oanquan.OAnQuan(**random_state)),
        "ideal_moves": ideal_moves,
    }

In [5]:
print("current index: ", index)

current index:  30


In [6]:
with open("challenges.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=['game', 'hint', 'ideal_moves', 'id'])
    writer.writeheader()
    for i in range(10):
        challenge = create_challenge(10)
        challenge["id"] = index
        index += 1
        challenge["game"] = json.dumps(challenge["game"])
        challenge["hint"] = json.dumps(challenge["hint"])
        writer.writerow(challenge)
